Classification with "deep" tensor product
JLF May 2018

In [ ]:
import os
import numpy as np
import pandas
import math
from random import randint

In [ ]:
# FUNCTIONS TO READ SEQUENCE AND ACTIVITY DATA
import Bio
from rdkit.Chem import AllChem
from rdkit.Chem.inchi import MolFromInchi

def getInchi(filename): 
    # Return a dictonary InChIs{MNX:Inchi,..}
    # Change this function if
    # ID/name is not in row[0] and InChI not in row[5]
    InChIs = {}
    with open(filename) as h:
        for line in h:
            if line.startswith('#'):
                continue
            row = line.rstrip().split('\t')
            mnx = row[0]
            InChIs[mnx] = row[5]
    return InChIs

def fasta_reader(filename):
  from Bio.SeqIO.FastaIO import FastaIterator
  with open(filename) as handle:
    for record in FastaIterator(handle):
      yield record

def get_seq(ID):
    # This function exract sequences from ebi
    command  = "curl -k -X GET --header 'Accept:text/x-fasta' 'https://www.ebi.ac.uk/proteins/api/proteins/" 
    command += str(ID) + "'"
    command += " > JLF-2018-tmp.fasta"
    os.system(command)
    s =""
    for entry in fasta_reader("JLF-2018-tmp.fasta"):
        s = str(entry.seq)
    return s

def read_data_file(data_file_name,chem_file_name):
    # Change based on the file format must contain
    # a chemical id, a sequence id and an activity
    # return molecules, sequences and activities
    N = 0
    EC = {}
    ID = {}
    CP = {}
    RX = {}
    KM = {}
    MOL = {}
    SEQ = {}
    InChIs = getInchi(chem_file_name)
    with open(data_file_name) as h:
        for line in h:
            row = line.rstrip().split('\t')
            seq = get_seq(row[1])
            mol = None
            if row[2] in InChIs:
                mol = MolFromInchi(InChIs[row[2]])
            if (len(seq) > 0) and (mol is not None): # skip empty entries
                EC[N] = row[0]
                ID[N] = row[1]
                CP[N] = row[2]
                RX[N] = row[3]
                KM[N] = row[4]
                SEQ[N] = seq
                MOL[N] = mol
                N += 1
    return KM, MOL, SEQ

In [ ]:
# FUNCTIONS TO COMPUTE FINGERPRINTS FOR CHEM AND PROTEINS
from sklearn.preprocessing import StandardScaler, normalize

# Get Chemical Fingerprints and k-mers
def get_ChemFp(mol,binary):
    if binary:
        molFp = AllChem.GetMorganFingerprintAsBitVect(mol,2,ChemFpSize)
        Fp = [int(x) for x in list(molFp.ToBitString())]
    else:
        molFp = AllChem.GetHashedMorganFingerprint(mol,2,ChemFpSize)
        Fp = list(molFp) 
    return Fp

# k-mers for protein
def kmer(sequence,k):
    length=len(sequence)
    if k>=length:
        return # if the kmer size if greater than the lenght of the sequence
    stepsize=k-1
    i=0
    kmers=[]
    while i+stepsize<length:
        s = sequence[i:i+k]
        r = s[::-1]
        if r < s:
            s = r # only the smalest one
        kmers.append(s)
        i+=1
    return set(kmers)

def kmerset(seq,length):
    n_seq = len(seq)
    if n_seq < 1:
        return # no sequences
    kmers = kmer(seq[0],length)
    for i in range(n_seq):
        s = kmer(seq[i],length)
        kmers.update(s)
        set(kmers)
    return(kmers)   

# one hot encoding for protein
DNA = 'ATCG '
PROTEIN = 'ACDEFGHIKLMNPQRSTUVWXY '
def seq_2_onehot(seq,alphabet):
    # define a mapping of chars to integers
    char_to_int = dict((c, i) for i, c in enumerate(alphabet))
    # integer encode input data
    integer_encoded = [char_to_int[char] for char in seq]
    # one hot encode
    onehot_encoded = list()
    for value in integer_encoded:
        letter = [0 for _ in range(len(alphabet))]
        letter[value] = 1
        onehot_encoded.append(letter)
    return(onehot_encoded)

def onehot_2_seq(onehot_encoded,alphabet):
    # invert encoding
    int_to_char = dict((i, c) for i, c in enumerate(alphabet))
    seq = ''
    for i in range (0,len(onehot_encoded)):
        inverted = int_to_char[np.argmax(onehot_encoded[i])]
        seq += str(inverted)
    return(seq)

# main function to compute fingerprints
def get_fingerprint(MOL,SEQ,ChemFpSize,ProtFpSize,
                    KmerSize,binary,onehot):
    SIZE = len(SEQ)
    if onehot:
        alphabet = PROTEIN
        onehot_dim=len(alphabet) 
        seqsize=0
        for i in range(SIZE):
            if len(SEQ[i]) > seqsize:
                seqsize= len(SEQ[i])
        seqsize = KmerSize*(int(seqsize/KmerSize)+1)
        ProtFpSize=int(seqsize*onehot_dim)
    else:
        onehot_dim = 0
        alphabet=kmerset(SEQ,KmerSize)
        if ProtFpSize <= 0:
            ProtFpSize = len(alphabet)
        # hash kmers in alphadic
        alphadic={x:randint(0, ProtFpSize-1) for x in alphabet} 
        
    # Get the fingerprint
    FP =  np.zeros( (SIZE, ProtFpSize) )
    FC =  np.zeros( (SIZE, ChemFpSize) )
    for i in range(SIZE):    
        FC[i] = get_ChemFp(MOL[i],binary)
        if onehot:
            s_hot = np.array(seq_2_onehot(SEQ[i],alphabet))
            hotsize = s_hot.shape[0]*s_hot.shape[1]
            s_hot = s_hot.reshape(hotsize)
            s = np.zeros( (ProtFpSize) )
            for k in range(hotsize):
                s[k] = s_hot[k]
            FP[i] = s
        else:
            for k in alphabet:
                FP[i][alphadic[k]] += SEQ[i].count(k)
        if binary == False:
            FC[i] = normalize(FC[i].reshape(1,-1))
            # FP[i] = normalize(FP[i].reshape(1,-1))
    return FC, FP, onehot_dim

In [ ]:
# DEEP LEARNING MODEL FUNCTIONS
from keras.models import Sequential, Model
from keras.layers import Input, Dense, LSTM, Conv1D, LocallyConnected1D
from keras.layers import Dropout, MaxPooling1D, Flatten, Merge, RepeatVector
from keras.layers import Merge, Lambda, Reshape, multiply, concatenate
from keras.layers.normalization import BatchNormalization
os.environ['KERAS_BACKEND'] = 'tensorflow'

def CROP(dimension, start, end):
    # Crops (or slices) a Tensor on a given dimension from start to end
    # example : to crop tensor x[:, :, 5:10]
    # call x = crop(2,5,10)(x) to slice the second dimension
    
    def func(x):
        if dimension == 0:
            return x[start: end]
        if dimension == 1:
            return x[:, start: end]
        if dimension == 2:
            return x[:, :, start: end]
        if dimension == 3:
            return x[:, :, :, start: end]
        if dimension == 4:
            return x[:, :, :, :, start: end]
    return Lambda(func)

def CONV(inputs, input_size, latent_size, ouput_size,
             strides, filters, dropout, hidden,
             activation, loss, optimizer): 
    out = Reshape((input_size,1)) (inputs) # needed for Conv1D
    out = LocallyConnected1D(filters=filters, 
                 kernel_size=latent_size, strides=strides,
                 activation=activation) (out)
#    out = Conv1D(filters=filters, 
#                 kernel_size=latent_size, strides=strides,
#                 activation=activation) (out)
#    if hidden:
#        out = Conv1D(filters=int(filters/10), 
#                     kernel_size=int(input_size/latent_size),
#                     activation=activation) (out)
    out = Flatten() (out)    
    out = Dropout(dropout) (out)
    outputs = Dense(ouput_size, 
#                    kernel_regularizer = 'l1',
                    activation=activation) (out)
    return outputs

def RNN(inputs, input_size, latent_size, ouput_size,
               dropout, hidden, activation, loss, optimizer):   

    timesteps=int(input_size/latent_size)
    out = Reshape((timesteps,latent_size)) (inputs) # needed for LSTM   
    out = LSTM(latent_size,activation=activation, return_sequences=True)(out)
    if hidden:
        out = LSTM(latent_size,activation=activation, return_sequences=True)(out)
    out = Flatten() (out)    
    out = Dropout(dropout) (out)
    outputs = Dense(ouput_size, 
#                    kernel_regularizer = 'l1',
                    activation=activation) (out)
    return outputs
 


def DENSE(inputs, input_size, latent_size, ouput_size,
               dropout, hidden, activation, loss, optimizer):   
    out = Dense(latent_size, 
#                kernel_regularizer = 'l1',
                activation=activation) (inputs)
    out = Dropout(dropout) (out)
    r = 1
    while r <= hidden:
        out = Dense(int(latent_size/2**r), 
#                    kernel_regularizer = 'l1',
                    activation=activation) (out)
        r += 1    
    outputs = Dense(ouput_size, 
#                    kernel_regularizer = 'l1',
                    activation=activation) (out)
    return outputs

def SEQCHEM(seq_model, che_model,
            seq_input_size, che_input_size, 
            seq_latent_size, che_latent_size, 
            mix_input_size,strides, filters, 
            dropout, hidden, 
            activation, activation_out, 
            loss, optimizer, metric):

    # split sequences from chemicals
    inputs = Input(shape=((seq_input_size+che_input_size,)))
    seq_inputs  = CROP(1,0,seq_input_size) (inputs)
    che_inputs = CROP(1,seq_input_size,seq_input_size+che_input_size) (inputs)
    # sequence model
    if seq_model == 'conv':
        seq = CONV(inputs=seq_inputs,
                   input_size=seq_input_size, latent_size=seq_latent_size, 
                   ouput_size=int(mix_input_size/2),
                   strides = strides, filters = filters, 
                   dropout=dropout, hidden=hidden, 
                   activation=activation, loss=loss, optimizer=optimizer)
    elif seq_model == 'dense':
        seq = DENSE(inputs=seq_inputs,
                    input_size=seq_input_size, latent_size=seq_latent_size,
                    ouput_size=int(mix_input_size/2),
                    dropout=dropout, hidden=hidden,
                    activation=activation, loss=loss, optimizer=optimizer)
    elif seq_model == 'rnn':
        seq = RNN(inputs=seq_inputs,
                    input_size=seq_input_size, latent_size=seq_latent_size,
                    ouput_size=int(mix_input_size/2),
                    dropout=dropout, hidden=hidden,
                    activation=activation, loss=loss, optimizer=optimizer)
    # chemical model
    if che_model == 'dense':
        che = DENSE(inputs=che_inputs,
                     input_size=che_input_size, latent_size=che_latent_size, 
                     ouput_size=int(mix_input_size/2),
                     dropout=dropout, hidden=hidden,
                     activation=activation, loss=loss, optimizer=optimizer)
    elif che_model == 'rnn':
        che = RNN(inputs=che_inputs,
                     input_size=che_input_size, latent_size=che_latent_size, 
                     ouput_size=int(mix_input_size/2),
                     dropout=dropout, hidden=hidden,
                     activation=activation, loss=loss, optimizer=optimizer)

    # complete mixed model
    out = concatenate([seq, che], axis=-1)
    out = Dense(mix_input_size, 
#                kernel_regularizer = 'l1',
                activation=activation) (out)
    outputs = Dense(1 ,activation=activation_out) (out)
    tensor = Model(inputs, outputs)
    tensor.compile(loss=loss,optimizer=optimizer,metrics=['accuracy'])
    print('Running Tensor model with', tensor.count_params(),'parameters' )
    return tensor

In [ ]:
# LOAD DATA (WARNING MAY TAKE A WHILE...)

datafile = '2.5.1_classification.tsv' 
chemfile = 'chem_prop.tsv' # This is the MetaNetX database
KM, MOL, SEQ = read_data_file(datafile,chemfile)
print("Loaded %d sequences, chemicals, and activity values" % len(SEQ))

In [ ]:
# COMPUTE FINGERPRINT 
# THAT'S WHERE YOU SET UP FINGERPRINT METHODS AND PARAMETERS

BINARY = False
ONEHOT = True
ChemFpSize = 1024
# the following 2 parameters are ingored if ONEHOT=True
ProtFpSize = 2048 # <= 0 for unfolded fingerprint
KMERSIZE = 3 

FC, FP, onehot_dim = get_fingerprint(MOL,SEQ,ChemFpSize,ProtFpSize,
                                     KMERSIZE,BINARY,ONEHOT)
print('SEQ  FP:    ', FP.shape)
print('CHEM FP:    ', FC.shape)
print('onehot dim: ', onehot_dim)

In [ ]:
# DIRECT TRAINING

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


seed = 1
np.random.seed(seed)
X = np.concatenate((FP,FC),axis=1)
Y =  np.zeros(len(SEQ),dtype=int) # classification
for i in range(len(SEQ)):
    Y[i]=int(KM[i])

print('sequence:',FP.shape,'+','chemical:',FC.shape,'=',X.shape,Y.shape)

# evaluate model with standardized dataset
model = KerasClassifier(build_fn=SEQCHEM, 
                        seq_model='conv', che_model='rnn',
                        seq_input_size=FP.shape[1], che_input_size=FC.shape[1], 
                        seq_latent_size=9*onehot_dim, che_latent_size=256, 
                        mix_input_size=8,
                        strides=3*onehot_dim, filters=8, 
                        dropout=0.25, hidden=0, 
                        activation='relu', activation_out='sigmoid', 
                        loss='binary_crossentropy', optimizer='adam', 
                        metric='accuracy',
                        epochs=100, batch_size=100, verbose=False)
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
results = cross_val_score(model, X, Y, cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
